In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
from collections import Counter
import cv2
import numpy as np
import random
import pickle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [4]:
train_dir = 'archive/train/'
test_dir = 'archive/test/'
ckplus_dir = 'CK+/'
jaffe_dir = 'jaffedbase/'

In [5]:
# Fungsi untuk memuat dataset
def load_dataset(img_dir):
    p = Path(img_dir)
    emotions = ['angry', 'happy', 'sad', 'neutral']  # Label sesuai dataset
    img_list = []
    
    for emotion in emotions:
        dir_path = p / emotion
        if dir_path.exists() and dir_path.is_dir():
            # Memilih file JPG dan PNG
            for file in dir_path.glob('*.jpg'):
                img = cv2.imread(str(file))
                if img is not None:
                    img_list.append((img, emotion))
            for file in dir_path.glob('*.png'):
                img = cv2.imread(str(file))
                if img is not None:
                    img_list.append((img, emotion))
    
    return img_list

def load_jaffe_dataset(img_dir):
    # Dictionary dengan kata kunci dari nama file dan label emosi
    target_labels = {
        "AN": "angry",
        "HA": "happy",
        "SA": "sad",
        "NE": "neutral"
    }
    img_list = []
    p = Path(img_dir)

    # Looping melalui file di folder dataset
    for filename in p.glob("*"):
        # Cek apakah file adalah gambar
        if filename.suffix in [".tiff", ".jpg", ".png"]:
            label = None
            # Tentukan label berdasarkan kata kunci dalam nama file
            for keyword, emotion in target_labels.items():
                if keyword in filename.stem:
                    label = emotion
                    break

            # Jika label ditemukan, muat gambar dan tambahkan ke list
            if label:
                img = cv2.imread(str(filename), cv2.IMREAD_GRAYSCALE)  # Memuat gambar dalam grayscale
                if img is not None:
                    img_list.append((img, label))

    return img_list

# Fungsi untuk memfilter dataset berdasarkan label yang diinginkan
def filter_labels(dataset, labels_to_include):
    filtered_data = []
    for img, label in dataset:
        if label in labels_to_include:
            filtered_data.append((img, label))
    return filtered_data

# Memuat dataset FER 13 dan CK+
# fer_train = load_dataset(train_dir)
# fer_test = load_dataset(test_dir)
ck_data = load_dataset(ckplus_dir)
jaffe_data = load_jaffe_dataset(jaffe_dir)

# Pilih gambar dengan label 'angry', 'sad', 'happy' dari CK+ dan 'neutral' dari FER
# fer_train_filtered = filter_labels(fer_train, ['neutral'])
# fer_test_filtered = filter_labels(fer_test, ['neutral'])
ck_filtered = filter_labels(ck_data, ['angry', 'sad', 'happy'])

# Gabungkan dataset
# data_images = ck_filtered + fer_train_filtered + fer_test_filtered
data_images = ck_filtered + jaffe_data

In [6]:
def preprocess(img_list):
    std_img_list = []
    for item in img_list:
        img = item[0]
        label = item[1]
        
        if label == 'angry' or label == 'AN':
            img_label = 0
        elif label == 'happy' or label == 'HA':
            img_label = 1
        elif label == 'neutral' or label == 'NE':
            img_label = 2
        elif label == 'sad' or label == 'SA':
            img_label = 3
        elif label == 'disgust' or label == 'DI':
            img_label = 4
        elif label == 'fear' or label == 'FE':
            img_label = 5
        elif label == 'surprise' or label == 'SU':
            img_label = 6
        std_img_list.append([img, img_label])
    return std_img_list
    
std_img_list = preprocess(data_images)

In [7]:
train_labels = [item[1] for item in std_img_list]
print(f'data label counts: {np.bincount(train_labels)}')

data label counts: [165 238  30 115]


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_img(img_list, size=(128, 128)):
    img_arr = []
    for img in img_list:
        img_pre = img[0]
        label = img[1]
        
        faces = face_cascade.detectMultiScale(img_pre, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            img_pre = img_pre[y:y+h, x:x+w]
            img_pre = cv2.cvtColor(img_pre, cv2.COLOR_BGR2GRAY) if len(img_pre.shape) > 2 else img_pre
            img_pre = cv2.resize(img_pre, size, interpolation=cv2.INTER_LANCZOS4)
            img_arr.append([img_pre, label])
    
    return img_arr

# Memproses dataset
data_pre = preprocess_img(std_img_list)

In [9]:
def balance_data(data_pre):
    # Hitung jumlah sampel untuk setiap label
    label_counts = Counter([data[1] for data in data_pre])
    max_count = max(label_counts.values())
    
    # Tempatkan data dalam dictionary berdasarkan label
    label_dict = {label: [] for label in label_counts.keys()}
    for img, label in data_pre:
        label_dict[label].append([img, label])
    
    balanced_data = []
    
    # Melakukan balancing dengan oversampling atau undersampling
    for label, images in label_dict.items():
        if len(images) < max_count:
            # Oversample: tambahkan data dengan cara sampling acak
            oversampled_images = random.choices(images, k=max_count)
            balanced_data.extend(oversampled_images)
        else:
            # Undersample: ambil data dengan sampling acak
            undersampled_images = random.sample(images, k=max_count)
            balanced_data.extend(undersampled_images)
    
    # Acak urutan data agar tidak urut berdasarkan label
    random.shuffle(balanced_data)
    
    return balanced_data

# Menyeimbangkan dataset
balanced_data = balance_data(data_pre)

In [10]:
print(f'size: {len(balanced_data)}')
print(f'labels: {np.bincount([item[1] for item in balanced_data])}')

size: 780
labels: [195 195 195 195]


In [11]:
def prepare_data(std_img_list, target_size=(128, 128)):
    images = []
    labels = []
    
    for img, label in std_img_list:
        # Pastikan gambar adalah grayscale (bukan RGB)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_LANCZOS4)
        
        if img.shape == (target_size[0], target_size[1]):
            images.append(img)
            labels.append(label)
        else:
            print(f"Warning: Skipping image with shape {img.shape}")  # Memberi tahu jika ada gambar yang gagal
        
    # Konversi images dan labels ke numpy array
    images = np.array(images, dtype=np.float32).reshape(-1, target_size[0], target_size[1], 1)  # Tambahkan dimensi channel
    labels = np.array(labels)
    
    # Normalisasi gambar
    images = images / 255.0 
    
    # Pastikan label adalah integer sebelum one-hot encoding
    labels = np.array(labels)
    
    # One-hot encode label
    labels = to_categorical(labels, num_classes=4)  # Sesuaikan jumlah kelas
    
    return images, labels

# Memproses dataset
images, labels = prepare_data(balanced_data)

In [12]:
print("Train images shape:", images.shape)

Train images shape: (780, 128, 128, 1)


In [13]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [14]:
# Membuat model CNN untuk ukuran input 128x128
model = Sequential()

# Layer 1
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D((2, 2)))

# Layer 2
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Layer 3
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(4, activation='softmax'))

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih Model
history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    epochs=30,
    batch_size=64
)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.3528 - loss: 1.5171 - val_accuracy: 0.5321 - val_loss: 1.1821
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.5283 - loss: 1.1007 - val_accuracy: 0.7372 - val_loss: 1.0444
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.6246 - loss: 1.0158 - val_accuracy: 0.8462 - val_loss: 0.6815
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8098 - loss: 0.6133 - val_accuracy: 0.8333 - val_loss: 0.5034
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8369 - loss: 0.4531 - val_accuracy: 0.8718 - val_loss: 0.4284
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.8970 - loss: 0.3762 - val_accuracy: 0.8526 - val_loss: 0.3323
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.9036 - loss: 0.2754 - val_accuracy: 0.9359 - val_loss: 0.2358
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9399 - loss: 0.1915 - val_accuracy: 0.9487 - val_loss:

In [15]:
# Evaluasi Model pada data uji
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.9849 - loss: 0.0679
Test accuracy: 0.9871794581413269


In [16]:
with open('modeldl.pkl', 'wb') as file:
    pickle.dump(model, file)

In [19]:
# After training the model, save it in .h5 format
model.save('modeldl.h5')  # You can also use .keras extension


In [12]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Fungsi untuk memuat dan memproses gambar
def preprocess_image(img_path, target_size=(128, 128)):
    try:
        # Membaca gambar
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Gambar tidak ditemukan atau tidak dapat dimuat.")
        
        # Deteksi wajah menggunakan Haarcascade
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        if len(faces) == 0:
            raise ValueError("Wajah tidak terdeteksi dalam gambar.")
        
        # Ambil wajah pertama yang terdeteksi
        x, y, w, h = faces[0]
        face = img[y:y+h, x:x+w]

        # Resize gambar ke ukuran target
        face_resized = cv2.resize(face, target_size, interpolation=cv2.INTER_LANCZOS4)

        # Normalisasi piksel dan tambahkan dimensi batch dan channel
        face_resized = face_resized.astype('float32') / 255.0
        face_resized = np.expand_dims(face_resized, axis=(0, -1))  # Tambahkan dimensi batch dan channel
        return face_resized
    except Exception as e:
        raise ValueError(f"Kesalahan dalam memproses gambar: {e}")

# Fungsi untuk memprediksi emosi dari gambar
def predict_emotion(img_path, model):
    # Proses gambar
    processed_img = preprocess_image(img_path)

    # Lakukan prediksi
    predictions = model.predict(processed_img)
    predicted_class = np.argmax(predictions)

    # Label emosi berdasarkan indeks
    emotions = {0: 'angry', 1: 'happy', 2: 'neutral', 3: 'sad'}
    return emotions.get(predicted_class, "Unknown")

# Muat model yang telah disimpan
model = load_model('modeldl.h5')
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Gambar yang ingin diuji
img_path = 'test_happy_nondataset.jpg'  # Ganti dengan path gambar Anda

# Melakukan prediksi
try:
    emotion = predict_emotion(img_path, model)
    print(f"Prediksi emosi untuk gambar: {emotion}")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Prediksi emosi untuk gambar: happy
